In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [8]:
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X.shape, y.shape

((1000, 20), (1000,))

In [9]:
# X = np.hstack([np.ones((1000, 1)), X]) # to add intercept

In [10]:
X.mean(axis=0), X.std(axis=0) # no need to standardize

(array([-0.03060796,  0.01948247,  0.01855188,  0.03244342,  0.03462413,
         0.03193399, -0.02155626,  0.01240107,  0.04735593, -0.00730769,
         0.01367596,  0.01885292,  0.03472163, -0.00434747, -0.03232739,
        -0.01466931,  0.02706135,  0.00597234,  0.01005371, -0.00861443]),
 array([0.96509178, 1.01583209, 0.97798208, 0.9966703 , 1.01065482,
        0.98071258, 0.96732736, 1.04320882, 1.04007165, 0.97859211,
        1.12901516, 1.01655085, 1.00878989, 0.98378539, 1.02168685,
        1.3216333 , 1.01699317, 1.29599272, 0.90793208, 0.98017205]))

## My Logistic Regression Class realization

### Features to add:
- <b>Different types of gradient descent (GD) </b>
    - Full GD (already implemented)
    - stochastic GD (SGD) one random object used to find gradient
    - stochastic average SAGD (several random objects used to find gradient)
    
- <b>Different loss functions </b>
    - Logloss (already implemented)
    - MSE :) for fun
- <b>Info about model </b>
    - Coefficients, intercept, parameters set (learning rate, regularization)
    - Stats (p-values, confidence intervals, model adequacy)
    - Quality metrics w
- <b> Marginal Effects (ME) </b>
        - ME for each factor
        - ME at means, median, at any factors given
        - average ME (AME) for each factor
- <b> Non-linear factors </b>
        - ME for them also

In [11]:
class Logreg:
    def __init__(self, learning_rate=0.05, iterations=1500, C=1.0):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.C = C
        self.weights = None
        self.intercept = None
        
        # history
        self.iters_list = []
        self.loss_list = []
    
    def fit(self, X, y):
        # number of observations
        n = len(y)
        # number of features
        k = X.shape[1]
        
        # 1. Initialize weights
        self.weights = np.zeros(k)
        self.intercept = 0
        
        for iteration in range(self.iterations):
            # 2 Predict
            z = np.dot(X, self.weights) + self.intercept
            y_hat = 1 / (1 + np.exp(-z))
            # 3 Calculate logloss
            #logloss = np.sum(-y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat))
            logloss = np.sum(np.log(1 + np.exp(- y * y_hat)))
        
            # 4 Derivative by weights
            derivative_weights = (1 / n) * np.dot(X.T, (y_hat - y)) # (a - y) * x
            derivative_intercept = (1 / n) * np.sum(y_hat - y)

            # 5 Update weights
            self.weights -= self.learning_rate * (derivative_weights + self.C * self.weights)
            self.intercept -= self.learning_rate * (derivative_intercept + self.C * self.intercept)
            
            #print('Iteration:', iteration, 'Total logLoss =', logloss)
            
            # history update
            self.iters_list.append(iteration)
            self.loss_list.append(logloss)
            # 6 Repeat
    

    def predict(self, X):
        z = np.dot(X, self.weights) + self.intercept
        pred = 1 / (1 + np.exp(-z))
        return np.array([1 if i > 0.5 else 0 for i in pred])
    
    def predict_proba(self, X):
        z = np.dot(X, self.weights) + self.intercept
        return 1 / (1 + np.exp(-z))

### Instantiate, Fit and Predict

In [12]:
logreg = Logreg(C=1.0)

In [13]:
logreg.fit(X, y)

In [14]:
mine_pred = logreg.predict(X)
np.unique(mine_pred)

array([0, 1])

## Sklearn

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
lr = LogisticRegression(C=1.0, solver='liblinear')

In [17]:
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr.intercept_

array([-0.55667614])

In [19]:
lr.coef_

array([[ 0.0051079 ,  0.18580036, -0.03000362,  0.10656345,  0.14646711,
        -0.12939623, -0.11166564,  0.12635051,  0.1195064 , -0.03171033,
         0.81794541, -0.03529246,  0.1252057 , -0.07348786, -0.09444035,
         0.31862608,  0.22912314,  2.92178327, -0.22983122,  0.08791236]])

In [20]:
pred = lr.predict(X)

In [21]:
y.shape, pred.shape

((1000,), (1000,))

In [22]:
np.unique(y), np.unique(pred)

(array([0, 1]), array([0, 1]))

## Comparison: Mine VS Sklearn

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [24]:
accuracy_score(y, pred), roc_auc_score(y, pred)

(0.908, 0.9080156320625283)

In [25]:
accuracy_score(y, mine_pred), roc_auc_score(y, mine_pred)

(0.89, 0.890063560254241)

In [26]:
logreg.weights

array([-0.01549807,  0.01859498,  0.00502167,  0.00140718,  0.02130198,
       -0.00631488, -0.01062253, -0.01900558,  0.00876403,  0.00103525,
        0.11285081, -0.00194511,  0.02070505, -0.01657789, -0.0211903 ,
        0.01629537,  0.01019341,  0.34379146, -0.01248403,  0.00437411])

### Marginal Effects

In [32]:
# for any linear factor it is just its weight

In [33]:
logreg.weights[0]

-0.015498069345919387